In [1]:
import numpy as np

np.show_config()

Build Dependencies:
  blas:
    detection method: pkgconfig
    found: true
    include directory: /opt/arm64-builds/include
    lib directory: /opt/arm64-builds/lib
    name: openblas64
    openblas configuration: USE_64BITINT=1 DYNAMIC_ARCH=1 DYNAMIC_OLDER= NO_CBLAS=
      NO_LAPACK= NO_LAPACKE= NO_AFFINITY=1 USE_OPENMP= SANDYBRIDGE MAX_THREADS=3
    pc file directory: /usr/local/lib/pkgconfig
    version: 0.3.23.dev
  lapack:
    detection method: internal
    found: true
    include directory: unknown
    lib directory: unknown
    name: dep4350285776
    openblas configuration: unknown
    pc file directory: unknown
    version: 1.26.4
Compilers:
  c:
    args: -fno-strict-aliasing, -DBLAS_SYMBOL_SUFFIX=64_, -DHAVE_BLAS_ILP64
    commands: cc
    linker: ld64
    linker args: -fno-strict-aliasing, -DBLAS_SYMBOL_SUFFIX=64_, -DHAVE_BLAS_ILP64
    name: clang
    version: 14.0.0
  c++:
    args: -DBLAS_SYMBOL_SUFFIX=64_, -DHAVE_BLAS_ILP64
    commands: c++
    linker: ld64
    linker

# side model

In [3]:
import pandas as pd

df_features = pd.read_parquet("data/features_1h.parquet")
label_side = np.load("data/label_side_1h.npy")
print(df_features.shape)
print(label_side.shape)

df_features.isna().sum().sort_values(ascending=False)

(44867, 11841)
(44867, 7)


4h_dft_dom_cycle_ddt_lag39    364
4h_dft_dom_cycle_ddt_lag38    360
4h_dft_dom_cycle_dt_lag39     360
4h_dft_dom_cycle_ddt_lag37    356
4h_dft_dom_cycle_lag39        356
                             ... 
1h_ac_23                        0
1h_ac_22                        0
1h_mod_stochastic               0
1h_trendflex                    0
1h_ac_31                        0
Length: 11841, dtype: int64

In [4]:
df_features = df_features[364:]
label_side = label_side[364:]

print(df_features.shape)
print(label_side.shape)

label_side

(44503, 11841)
(44503, 7)


array([[ 1.58666418e+12,  6.81173000e+03,  6.91519000e+03, ...,
         6.77566000e+03,  7.38005150e+04,  1.00000000e+00],
       [ 1.58670006e+12,  6.91519000e+03,  7.08500000e+03, ...,
         6.90500000e+03,  7.22904840e+04,  1.00000000e+00],
       [ 1.58670972e+12,  7.08500000e+03,  7.11225000e+03, ...,
         7.04632000e+03,  7.13427380e+04,  1.00000000e+00],
       ...,
       [ 1.73826822e+12,  1.05683000e+05,  1.05074100e+05, ...,
         1.04800000e+05,  4.94897800e+03, -1.00000000e+00],
       [ 1.73826996e+12,  1.05074000e+05,  1.05118400e+05, ...,
         1.04737000e+05,  4.89163800e+03, -1.00000000e+00],
       [ 1.73827302e+12,  1.05118500e+05,  1.04676900e+05, ...,
         1.04628300e+05,  4.84562100e+03, -1.00000000e+00]])

In [5]:
from custom_indicators.config import SIDE_ALL

side_features = df_features[SIDE_ALL]
print(side_features.shape)
side_label = (label_side[:, 6].astype(int) == 1).astype(int)
print(np.unique(side_label, return_counts=True))

train_test_split_point = int(side_features.shape[0] * 0.8)
side_features_train = side_features[:train_test_split_point]
side_features_test = side_features[train_test_split_point:]
side_label_train = side_label[:train_test_split_point]
side_label_test = side_label[train_test_split_point:]

print(np.unique(side_label_test, return_counts=True))

(44503, 2368)
(array([0, 1]), array([21519, 22984]))
(array([0, 1]), array([4239, 4662]))


In [6]:
import lightgbm as lgb
import optuna
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


def objective(trial):
    params = {
        "objective": "binary",
        "metric": "auc",
        "num_threads": -1,
        "verbose": -1,
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        # "early_stopping_round": 100,
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 300),
        "max_depth": trial.suggest_int("max_depth", 30, 100),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 300),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-4, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-4, 100),
    }
    dtrain = lgb.Dataset(side_features_train, side_label_train)
    dtest = lgb.Dataset(side_features_test, side_label_test)
    model = lgb.train(
        params,
        dtrain,
        # valid_sets=[dtest],
        num_boost_round=trial.suggest_int("num_boost_round", 100, 1000),
    )
    pred_proba = model.predict(side_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(side_label_test, pred_label)
    recall = recall_score(side_label_test, pred_label)
    f1 = f1_score(side_label_test, pred_label)
    auc = roc_auc_score(side_label_test, pred_proba)
    print(f"precision: {precision}, recall: {recall}, f1: {f1}")
    return auc


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, n_jobs=1)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-03-11 12:37:58,260] A new study created in memory with name: no-name-82628b5f-f0bf-4f8e-a7d1-5279201bd8d0
[I 2025-03-11 12:38:17,857] Trial 0 finished with value: 0.8815023698250876 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 98, 'max_depth': 53, 'min_gain_to_split': 0.14130952657166435, 'min_data_in_leaf': 74, 'lambda_l1': 30.276043688463176, 'lambda_l2': 45.05264392629679, 'num_boost_round': 689}. Best is trial 0 with value: 0.8815023698250876.


precision: 0.8006939926263283, recall: 0.7919347919347919, f1: 0.7962903051871023


[I 2025-03-11 12:38:26,621] Trial 1 finished with value: 0.8803656553125767 and parameters: {'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 293, 'max_depth': 51, 'min_gain_to_split': 0.4389819126413067, 'min_data_in_leaf': 25, 'lambda_l1': 17.198747209641244, 'lambda_l2': 85.46162048481374, 'num_boost_round': 412}. Best is trial 0 with value: 0.8815023698250876.


precision: 0.7972827259003666, recall: 0.793007293007293, f1: 0.7951392622862673


[I 2025-03-11 12:38:50,170] Trial 2 finished with value: 0.879610982937239 and parameters: {'extra_trees': False, 'boosting': 'dart', 'num_leaves': 260, 'max_depth': 62, 'min_gain_to_split': 0.04128954193452784, 'min_data_in_leaf': 203, 'lambda_l1': 59.241665119998586, 'lambda_l2': 56.59953768805428, 'num_boost_round': 518}. Best is trial 0 with value: 0.8815023698250876.


...

precision: 0.7985766659478111, recall: 0.7942942942942943, f1: 0.7964297236261964


[I 2025-03-11 13:10:41,563] Trial 98 finished with value: 0.8818316850871699 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 258, 'max_depth': 88, 'min_gain_to_split': 0.1386715908501006, 'min_data_in_leaf': 70, 'lambda_l1': 26.33394872387798, 'lambda_l2': 6.094803899333474, 'num_boost_round': 843}. Best is trial 94 with value: 0.8829532697190163.


precision: 0.8007820986313274, recall: 0.7906477906477907, f1: 0.7956826767404209


[I 2025-03-11 13:11:03,126] Trial 99 finished with value: 0.8823747415396389 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 251, 'max_depth': 82, 'min_gain_to_split': 0.24786618287018666, 'min_data_in_leaf': 20, 'lambda_l1': 21.619585855102354, 'lambda_l2': 53.433290919082964, 'num_boost_round': 712}. Best is trial 94 with value: 0.8829532697190163.


precision: 0.7992227979274611, recall: 0.7940797940797941, f1: 0.7966429954809554


In [7]:
study.best_params

{'extra_trees': True,
 'boosting': 'dart',
 'num_leaves': 280,
 'max_depth': 79,
 'min_gain_to_split': 0.02384212783734981,
 'min_data_in_leaf': 37,
 'lambda_l1': 19.970715976805124,
 'lambda_l2': 6.591928341724914,
 'num_boost_round': 794}

In [8]:
params = {
    # "is_unbalance": True,
    **study.best_params,
}

dtrain = lgb.Dataset(side_features_train, side_label_train)
model = lgb.train(params, dtrain)

In [9]:
import plotly.express as px

res = model.predict(side_features_train)

fig = px.histogram(res, nbins=100)
fig.show()

In [10]:
import plotly.express as px

res = model.predict(side_features_test)
print(roc_auc_score(side_label_test, res))

fig = px.histogram(res, nbins=100)
fig.show()

0.8809698385069933


In [11]:
model.save_model("custom_indicators/models/model_side.txt")

# meta model

In [15]:
import lightgbm as lgb
import numpy as np
import pandas as pd

from custom_indicators.config import SIDE_ALL

df_features = pd.read_parquet("data/features_25m.parquet")
meta_label = np.load("data/label_meta_25m.npy")

model_side = lgb.Booster(model_file="custom_indicators/models/model_side.txt")
side_model_res = model_side.predict(df_features[SIDE_ALL])
df_features["side_model_res"] = side_model_res

df_features.isna().sum(axis=0).sort_values(ascending=False)

2h_voss_ddt_lag9                   808
2h_adaptive_stochastic_ddt_lag5    808
2h_adaptive_stochastic_ddt_lag3    808
2h_adaptive_stochastic_ddt_lag2    808
2h_adaptive_stochastic_ddt_lag1    808
                                  ... 
25m_comb_spectrum_pwr_14             0
25m_comb_spectrum_pwr_13             0
25m_comb_spectrum_pwr_12             0
25m_comb_spectrum_pwr_11             0
side_model_res                       0
Length: 11842, dtype: int64

In [16]:
df_features = df_features[808:]
meta_label = meta_label[808:]

print(df_features.shape)

assert df_features.shape[0] == meta_label.shape[0]
print(meta_label.shape)

(105366, 11842)
(105366, 7)


In [17]:
from custom_indicators.config import META_ALL

meta_features = df_features[META_ALL]
print(meta_features.shape)
meta_label = meta_label[:, 6].astype(int)

train_test_split_point = int(meta_features.shape[0] * 0.8)
meta_features_train = meta_features[:train_test_split_point]
meta_features_test = meta_features[train_test_split_point:]
meta_label_train = meta_label[:train_test_split_point]
meta_label_test = meta_label[train_test_split_point:]

np.unique(meta_label_test, return_counts=True)

(105366, 2368)


(array([0, 1]), array([ 4740, 16334]))

In [19]:
import lightgbm as lgb
import optuna
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


def objective(trial):
    params = {
        "objective": "binary",
        "is_unbalance": True,
        "num_threads": -1,
        "verbose": -1,
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 300),
        "max_depth": trial.suggest_int("max_depth", 30, 100),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 300),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 100),
    }
    dtrain = lgb.Dataset(meta_features_train, meta_label_train)
    dtest = lgb.Dataset(meta_features_test, meta_label_test)
    model = lgb.train(
        params,
        dtrain,
        num_boost_round=trial.suggest_int("num_boost_round", 100, 1000),
    )
    pred_proba = model.predict(meta_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(meta_label_test, pred_label)
    recall = recall_score(meta_label_test, pred_label)
    f1 = f1_score(meta_label_test, pred_label)
    auc = roc_auc_score(meta_label_test, pred_proba)
    print(f"precision: {precision}, recall: {recall}, f1: {f1}, auc: {auc}")
    return f1


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, n_jobs=1)

[I 2025-03-09 00:39:46,143] A new study created in memory with name: no-name-cc442e5d-1385-4384-9b36-c30187edd3b6
[I 2025-03-09 00:40:46,500] Trial 0 finished with value: 0.8253643410852713 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 211, 'max_depth': 58, 'min_gain_to_split': 0.2209742462937076, 'min_data_in_leaf': 115, 'lambda_l1': 22.654427355888263, 'lambda_l2': 10.63943093000907, 'num_boost_round': 262}. Best is trial 0 with value: 0.8253643410852713.


precision: 0.8362025634581554, recall: 0.8148034774090853, f1: 0.8253643410852713, auc: 0.7604377294855957


[I 2025-03-09 00:42:05,340] Trial 1 finished with value: 0.7487705947808607 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 189, 'max_depth': 36, 'min_gain_to_split': 0.24431397988001136, 'min_data_in_leaf': 103, 'lambda_l1': 92.33619278791869, 'lambda_l2': 2.108174289446243, 'num_boost_round': 535}. Best is trial 0 with value: 0.8253643410852713.


precision: 0.9049891540130152, recall: 0.638545365495286, f1: 0.7487705947808607, auc: 0.7677831026271725


[I 2025-03-09 00:43:14,698] Trial 2 finished with value: 0.7603645777919636 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 45, 'max_depth': 92, 'min_gain_to_split': 0.26519168284752215, 'min_data_in_leaf': 205, 'lambda_l1': 7.046217306068517, 'lambda_l2': 6.031167550422434, 'num_boost_round': 671}. Best is trial 0 with value: 0.8253643410852713.


...

precision: 0.827551081550722, recall: 0.8455369168605363, f1: 0.836447324591951, auc: 0.7618946449615335


[I 2025-03-09 07:47:35,157] Trial 98 finished with value: 0.8563759922334533 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 227, 'max_depth': 96, 'min_gain_to_split': 0.027501967043091026, 'min_data_in_leaf': 179, 'lambda_l1': 0.0044415949969807515, 'lambda_l2': 1.9453925601760353, 'num_boost_round': 812}. Best is trial 98 with value: 0.8563759922334533.


precision: 0.8024400684931506, recall: 0.9180849761234235, f1: 0.8563759922334533, auc: 0.7600784700598633


[I 2025-03-09 07:51:03,300] Trial 99 finished with value: 0.8556810061454909 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 225, 'max_depth': 95, 'min_gain_to_split': 0.030261869122553776, 'min_data_in_leaf': 183, 'lambda_l1': 0.0074683325057127575, 'lambda_l2': 1.092236140255359, 'num_boost_round': 925}. Best is trial 98 with value: 0.8563759922334533.


precision: 0.8025306954050722, recall: 0.9163707603771275, f1: 0.8556810061454909, auc: 0.759422852283477


In [20]:
study.best_params

{'extra_trees': False,
 'boosting': 'gbdt',
 'num_leaves': 227,
 'max_depth': 96,
 'min_gain_to_split': 0.027501967043091026,
 'min_data_in_leaf': 179,
 'lambda_l1': 0.0044415949969807515,
 'lambda_l2': 1.9453925601760353,
 'num_boost_round': 812}

In [21]:
params = {
    "is_unbalance": True,
    **study.best_params,
}

model = lgb.train(
    params,
    lgb.Dataset(meta_features_train, meta_label_train),
)

In [22]:
import plotly.express as px

res = model.predict(meta_features_test)

fig = px.histogram(res, nbins=100)
fig.show()

In [23]:
model.save_model("custom_indicators/models/model_meta.txt")